In [ ]:
#————————————————————

# Name: OpenAI Assistant (V1)

# Purpose:

# This notebook will create a GPT Assistant data engineer using the OpenAI API, give it a data set, ask it to process it using it's instructions, and then prepare the resulting dataframe for download. The idea here is to test if the assistant can successfully complete data prep and some data engineering, then reduce the feature set using a logistic regression with LASSO regularization, subsetting only non-zero LASSO coefficient features in the final dataset. 

# This notebook will create a GPT Assistant using OpenAI's API and provide it with the training dataframe returned by the data engineer Assistant and a set of instructions to creating an "Extra Trees" Random Forest. Basic outline of instructions for the modeler:

# 1. Load the provided dataframe into a pandas df.
# 2. Split the data set into training and testing using a 75:25 split.
# 3. Train an Extra Trees random forest with 2000 trees.
# 4. Use the testing data to measure the model's accuracy, presicion, recall, and generate a confusion matrix.
# 5. Return the results in a single csv table. 

# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch)
# Create for: SDSC 2024
# Date Created: 22.01.2024
# Last Updated: 22.01.2024
# Python Version: 3.10.4

#General Sources:
#https://platform.openai.com/docs/api-reference?lang=python
#https://medium.com/ai-advances/complete-process-for-fine-tuning-gpt-3-5-turbo-using-openai-api-db4a50b3de1a

#Openai Usage:
#https://platform.openai.com/usage

#Additionals:
# - https://platform.openai.com/docs/models
# - https://openai.com/pricing

# Download Python packages (run the below command in terminal if packages have not yet been installed)
#pip install -r C:\Python\openai-lab\support\requirements\requirements.txt

#————————————————————

In [30]:
# Import required libraries
import os
import io
import time
from io import StringIO
import json
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import pandas as pd
import openai
from openai import OpenAI

In [31]:
# Load OpenAIKey
load_dotenv(dotenv_path=Path("C:\Python\openai-lab\.venv\.env"))
API_KEY = os.environ['OPENAI_API_KEY']

# Initalise Large Language Model (LLM)

client = OpenAI(api_key = API_KEY)

In [32]:
## Define Functions

def read_and_save_file(first_file_id, file_name):    
    # its binary, so read it and then make it a file like object
    file_data = client.files.content(first_file_id)
    file_data_bytes = file_data.read()
    file_like_object = io.BytesIO(file_data_bytes)
    #now read as csv to create df
    returned_data = pd.read_csv(file_like_object)
    returned_data.to_csv(file_name, index=False)
    return returned_data
    # file = read_and_save_file(first_file_id, "analyst_output.csv")
    
def files_from_messages(messages, asst_name):
    first_thread_message = messages.data[0]  # Accessing the first ThreadMessage
    message_ids = first_thread_message.file_ids
    print(message_ids)
    # Loop through each file ID and save the file with a sequential name
    for i, file_id in enumerate(message_ids):
        file_name = f"{asst_name}_output_{i+1}.csv"  # Generate a sequential file name
        read_and_save_file(file_id, file_name)
        print(f'saved {file_name}')    

In [33]:
# Load data

path_input = r"C:\Python\data\openfoodfacts.csv" #Change path if required
#df = pd.read_csv(path_input , sep='\t')

In [34]:
# create the assistant and give it the CSV file

mls = '''
You are a senior data engineer who will work with data in a csv file in your files. 
When the user asks you to perform your actions, use the csv file to read the data into a pandas dataframe.
You will perform data cleansing and transformation steps. The data set is to be used for a classification model.
Execute each of the steps listed below in your ACTIONS section. The user will identify the target variable. 

ACTIONS:

1. Read the file data into a pandas DataFrame by tab separated values.
2. Keep only the columns "product_name" and "categories_tags".
3. Filter the column "categories_tags" and only show values that contain 3 commas.
4. Remove empty or NA rows from columns "product_name" and "categories_tags".
5. Remove rows with non-english words from columns "product_name" and "categories_tags".
6. Replace the characters "en:" in column "categories_tags" with blank.
7. Remove stop words and adjectives from column "product_name".
8. Remove duplicate values from column "product_name".
9. Deduplicate the column "product_name" using an optimized method of fuzzy string matching. Similarly named rows should only appear once.
10. Trim and lowercase the values of all columns and prepare the results as Table_1.
11. Prepare Table_1 as a csv file for download by the user. 
12. Provide a summary paragraph explaining the preparation of the data set.

DO NOT:
1. Do not return any images. 
2. Do not return any other file types.
'''

# send the csv file to the assistant purpose files
response = client.files.create(
  file=open(path_input, "rb"),
  purpose="assistants"
)
print(response)
file__id = response.id

openai_assistant = client.beta.assistants.create(
    instructions=mls,
    name="data_assistant",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview", # gpt-4
    file_ids=[file__id]
)

# get the file id
fileId = openai_assistant.file_ids[0]
print(openai_assistant)

FileObject(id='file-mdjvXQoyO0Upxe7LmLRTBmNs', bytes=291784575, created_at=1706088798, filename='openfoodfacts.csv', object='file', purpose='assistants', status='processed', status_details=None)
Assistant(id='asst_uggm5vBAiO5NRdfwPyW8cbtv', created_at=1706088804, description=None, file_ids=['file-mdjvXQoyO0Upxe7LmLRTBmNs'], instructions='\nYou are a senior data engineer who will work with data in a csv file in your files. \nWhen the user asks you to perform your actions, use the csv file to read the data into a pandas dataframe.\nYou will perform data cleansing and transformation steps. The data set is to be used for a classification model.\nExecute each of the steps listed below in your ACTIONS section. The user will identify the target variable. \n\nACTIONS:\n\n1. Read the file data into a pandas DataFrame by tab separated values.\n2. Keep only the columns "product_name" and "categories_tags".\n3. Filter the column "categories_tags" and only show values that contain 3 commas.\n4. Rem

In [35]:
# make the request to the assistant

message_string = "Please execute your ACTIONS on the data stored in the csv file " + fileId + " . The Target variable is categories_tags"
print(message_string)

# Step 2: Create a Thread
thread = client.beta.threads.create()

# Step 3: Add a Message to a Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content= message_string
)

# Step 4: Run the Assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=openai_assistant.id
    #instructions="Overwrite hard-coded instructions here"
)

print(run.model_dump_json(indent=4))

while True:
    sec = 60
    # Wait for 5 seconds
    time.sleep(sec)  
    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(f'{sec} seconds later...')
    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            try:
                content = msg.content[0].text.value
                print(f"{role.capitalize()}: {content}")
            except AttributeError:
                # This will execute if .text does not exist
                print(f"{role.capitalize()}: [Non-text content, possibly an image or other file type]")
        break

Please execute your ACTIONS on the data stored in the csv file file-mdjvXQoyO0Upxe7LmLRTBmNs . The Target variable is categories_tags
{
    "id": "run_ApuIO20zElhRDsfOUYSzNHMN",
    "assistant_id": "asst_uggm5vBAiO5NRdfwPyW8cbtv",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1706088868,
    "expires_at": 1706089468,
    "failed_at": null,
    "file_ids": [
        "file-mdjvXQoyO0Upxe7LmLRTBmNs"
    ],
    "instructions": "\nYou are a senior data engineer who will work with data in a csv file in your files. \nWhen the user asks you to perform your actions, use the csv file to read the data into a pandas dataframe.\nYou will perform data cleansing and transformation steps. The data set is to be used for a classification model.\nExecute each of the steps listed below in your ACTIONS section. The user will identify the target variable. \n\nACTIONS:\n\n1. Read the file data into a pandas DataFrame by tab separated values.\n2. Keep only the columns \"product_name\" 

In [36]:
# extract the file names from the response and retrieve the content
asst_name = 'data_assistant'        
files_from_messages(messages, asst_name)

['file-WHTNGh9hAYjDmdeNnqwcmZ6B']
saved data_assistant_output_1.csv


In [28]:
df1 = pd.read_csv('data_assistant_output_1.csv')
display(df1)

# df2 = pd.read_csv('engineer_output_2.csv')
# display(df2)

# df3 = pd.read_csv('engineer_output_3.csv')
# display(df3)

,product_name,Table_1
0,apple turnover,"snacks,sweet-snacks,viennoiseries,apple-turnovers"
1,bread,"plant-based-foods-and-beverages,plant-based-fo..."
2,farmer cheese,"dairies,fermented-foods,fermented-milk-product..."
3,german fine bread,"plant-based-foods-and-beverages,plant-based-fo..."
4,"today's temptations, lithuanian rye bread","plant-based-foods-and-beverages,plant-based-fo..."
...,...,...
11227,"special k, crunch hot cereal, maple brown suga...","plant-based-foods-and-beverages,plant-based-fo..."
11228,"kellogg's, knourish, hot cereal, cinnamon, rai...","plant-based-foods-and-beverages,plant-based-fo..."
11229,"hot cereal, cranberry, almond","plant-based-foods-and-beverages,plant-based-fo..."
11230,kellogg's special k granola bars greek yogurt ...,"snacks,sweet-snacks,bars,cereal-bars"


In [ ]:
response = client.beta.assistants.delete(openai_assistant.id)
print(response)

In [ ]:
# load and check the file for the engineer
asst_file = 'engineer_output_1.csv'
df = pd.read_csv(asst_file)

display(df)

In [ ]:
# create the assistant and give it the CSV file

mls = '''
You are a data scientist who will build a predictive model with data from two csv files uploaded to your files. 
When the user asks you to perform your actions, use the csv file to read the data into a pandas dataframe.
Then continue with each of the steps listed below in you ACTIONS. The user will identify the target variable. 

ACTIONS:

1. Load the engineer_output_1 csv file into a pandas df of the same name.
2. Split the data set into training and testing data sets with a 25% split.
3. Train an Extra Trees random forest with 2000 trees
4. Use the testing data to measure the models accuracy, presicion, recall, and confusion matrix.
5. Format the testing data results as a csv table and prepare it for download by the user. 

DO NOT:
1. Return any images. 
'''

# send the csv file to the assistant purpose files
response = client.files.create(
  file=open(asst_file, "rb"),
  purpose="assistants"
)
print(response)
file_1_id = response.id

my_assistant = client.beta.assistants.create(
    instructions=mls,
    name="modeler_1",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview", # gpt-4
    file_ids=[file_1_id] # multiple files: file_ids=[file_1_id, file_2_id]
)

# get the file id
fileId = my_assistant.file_ids[0]
print(my_assistant)

In [ ]:
# make the request to the assistant

message_string = "Please execute your ACTIONS on " + fileId + " and prepare the resulting table for csv download. The Target variable is Class"
print(message_string)

# Step 2: Create a Thread
thread = client.beta.threads.create()

# Step 3: Add a Message to a Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content= message_string
)

# Step 4: Run the Assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=my_assistant.id
    #instructions="Overwrite hard-coded instructions here"
)

print(run.model_dump_json(indent=4))

while True:
    # Wait in between tries
    sec = 60
    time.sleep(sec)  
    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print('One eternity later...')
    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            try:
                content = msg.content[0].text.value
                print(f"{role.capitalize()}: {content}")
            except AttributeError:
                # This will execute if .text does not exist
                print(f"{role.capitalize()}: [Non-text content, possibly an image or other file type]")
        break

In [ ]:
asst_name = 'modeler'        
files_from_messages(messages, asst_name)

In [ ]:
df1 = pd.read_csv('modeler_output_1.csv')
display(df1)

In [ ]:
# Clean up the assistant

response = client.beta.assistants.delete(my_assistant.id)
print(response)